In [1]:
# Author: Jaehun Kim
# Email: rlawogns1204@unist.ac.kr
# Affiliation: UNIST BME BCILAB
# Date: 2023-06-12
#
# This code implements a tactile information processing model using a spiking
# neural network (SNN). It simulates the processing of tactile information from
# mechanoreceptors in the skin through primary afferent fibers (PA), cuneate nucleus
# neurons (PN and IN), and ultimately, somatosensory cortex neurons. The model
# incorporates lateral inhibition and various receptive field properties to
# represent a realistic processing of touch stimuli.

# with DIGIT-sensor
# pip install torch==1.8.1+cu101 torchvision==0.9.1+cu101 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import scipy.io
import time
import numpy as np
from PIL import Image
import io
import datetime
import cv2
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

from Function.snn_IZHIlayers import *
from Function.snn_plot import *
from Function.snn_simulation import *
from Function.snn_stimulation import *
from Function.snn_receptive_field_weights import *

from line_profiler import LineProfiler

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Check if GPU is available and print the CUDA version
print(f"GPU available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
# Enable GPU support if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Set the device to 'cpu'
device = 'cpu'
print(f"Using device: {device}")
# Start of the SNN (Spiking Neural Network) code

GPU available: True
CUDA version: 10.1
Using device: cpu


In [3]:
# Set sensor dimensions (height and width in millimeters)
sensor_h, sensor_w = 19, 16
# Set pixel dimensions (number of pixels in height and width)
# pixel_h, pixel_w = 320, 240///
pixel_h, pixel_w = 64,48
# Set image frames per second (FPS) of tactile sensor
image_FPS = 60

In [4]:
R = ReceptiveFieldWeights(pixel_h, pixel_w, device, type_output = 'speed')

speed
torch.Size([88, 3072])
sa_rf shape: torch.Size([88, 3072]) with height = 11 with width = 8
ra_rf shape: torch.Size([130, 3072]) with height = 13 with width = 10
sa_cn_pn_rf shape:  torch.Size([54, 88]) sa_cn_pn_step_height: 9 sa_cn_pn_step_width: 6
sa_cn_in_rf shape:  torch.Size([54, 88]) sa_cn_in_step_height: 9 sa_cn_in_step_width: 6
ra_cn_pn_rf shape:  torch.Size([88, 130]) ra_cn_pn_step_height: 11 ra_cn_pn_step_width: 8
ra_cn_in_rf shape:  torch.Size([88, 130]) ra_cn_in_step_height: 11 ra_cn_in_step_width: 8
sa_intopn_rf shape:  torch.Size([54, 54])
ra_intopn_rf shape:  torch.Size([88, 88])
cn_pn_sa_rf shape:  torch.Size([28, 54]) cn_pn_sa_rf_step_height: 7 cn_pn_sa_rf_step_width: 4
cn_in_sa_rf shape:  torch.Size([28, 54]) cn_in_sa_rf_step_height: 7 cn_in_sa_rf_step_width: 4
cn_pn_ra_rf shape:  torch.Size([28, 88]) cn_pn_ra_rf_step_height: 7 cn_pn_ra_rf_step_width: 4
cn_in_ra_rf shape:  torch.Size([28, 88]) cn_in_ra_rf_step_height: 7 cn_in_ra_rf_step_width: 4
cn_intopn_rf shap

In [ ]:
num_speeds = 19
num_stim = 1000
F = 30
pixel_h = 64
pixel_w = 48
device = 'cpu'  # Or your device of choice
data = []
labels = []

    # Generate the stimuli for each angle from 0 to 180 degrees
for speed in range(num_speeds):
# for speed in [18]:
    print(speed)
    # Generate the stimulus
    stimulus =  generate_speed_bar_stimuli(speed = speed, show_frames = [999])
    
    S = SNN(R, device = device, noise_std_val = 0)
    S.feedforward(stimulus)
    mean_firing_rates = plot_SNN(S,layers = ['sa','ra','cn'],plot_figure = True)



In [10]:
num_trials = 20
start_speed = 10
num_speeds = 20
num_stim = 1000
F = 10
pixel_h = 64
pixel_w = 48
device = 'cpu'  # Or your device of choice
data = []
labels = []

# Run the trials
for trial in range(num_trials):
    # Generate the stimuli for each speed from 10 to 19
    for speed in range(start_speed, start_speed + num_speeds):
        print(speed)
        # Generate the stimulus
        stimulus =  generate_speed_bar_stimuli(speed = speed, show_frames = [])
        
        S = SNN(R, device = device, noise_std_val = 0)
        S.feedforward(stimulus)
        mean_firing_rates = plot_SNN(S,plot_figure = False)

        # Save the data and labels
        data.append(mean_firing_rates.cpu().numpy())
        labels.append(speed)

print(labels)

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
11
12
13
14
15
16
17
18
19
20
21
22
2

In [12]:
# Convert data and labels to numpy arrays

data = np.array(data)
labels = np.array(labels)

# np.save('data_svm_speed.npy', data)
# np.save('labels_svm_speed.npy', labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Create and train the SVM
clf = svm.SVC(kernel='linear')  # You can change the kernel as needed
clf.fit(X_train, y_train)

# Test the SVM
y_pred = clf.predict(X_test)

# Print the accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

print(X_train.shape)
print(X_test.shape)
print(y_train)
print(y_test)

Accuracy: 1.0
(320, 28)
(80, 28)
[13 28 12 20 24 13 18 11 18 20 23 25 28 10 10 24 23 10 29 25 17 20 26 16
 23 17 29 27 25 19 21 17 22 16 27 10 29 15 15 11 27 19 27 17 14 25 21 22
 17 15 21 15 28 27 29 24 19 15 28 17 26 29 21 25 26 14 29 21 15 13 16 23
 25 20 20 22 13 11 26 28 25 18 16 18 23 19 27 13 17 18 22 19 13 26 13 16
 27 28 18 13 27 14 12 12 17 28 14 29 27 25 15 16 21 16 26 10 27 24 24 12
 13 13 13 21 12 19 28 21 15 15 12 22 29 17 29 27 18 27 14 17 12 14 23 26
 24 29 27 22 12 14 25 24 27 16 10 27 14 10 28 20 29 17 24 12 19 13 19 17
 19 26 23 14 14 20 14 17 15 15 12 16 29 22 25 18 20 12 28 22 11 25 21 10
 28 14 28 11 20 16 24 21 14 26 28 10 16 11 23 21 28 25 11 17 22 17 10 26
 17 22 20 10 18 12 20 28 12 25 18 18 14 10 24 26 10 16 13 26 29 16 26 17
 22 25 14 21 20 15 13 19 11 11 13 27 20 19 29 15 23 11 19 10 15 24 13 21
 29 22 15 14 11 23 25 18 23 16 18 23 24 16 13 29 13 24 20 24 19 27 17 19
 28 18 14 25 22 11 23 10 26 21 23 13 27 18 19 20 21 29 29 22 17 20 24 11
 29 10 18 21 16 20